In [62]:
## This file contains a prototype idea of trying to learn a value function that represents the viable region
## for a LIP model. 
## Author : Avadesh Meduri
## Date : 23/03/2020

import numpy as np
import IPython
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

In [63]:
'''
This is an inverted pendulum enviromment designed to use for RL. The action in this space is the step length
with respect to the current cop location. 9 possible actions are allowed including 0 step length. 
'''

class lipm_env:
    def __init__(self, h, max_step_length):
        self.omega = np.sqrt(9.81/h)
        self.max_leg_length = 0.3
        self.dt = 0.001
        self.h = h
        assert (np.linalg.norm([max_step_length, self.h]) < self.max_leg_length)
        self.action_space = np.linspace(-max_step_length, max_step_length, 9) #discretize to 9 actions
        self.A = np.matrix([[1, self.dt], [(self.omega**2)*self.dt, 1]])
        self.B = np.matrix([0, -(self.omega**2)*self.dt])
        self.t = 0
                                     
    def integrate_lip_dynamics(self, x_t, u_t):
        ## integrates dynamics for one time step
        assert np.shape(x_t) == (2,)
        x_t_1 = np.matmul(self.A, np.transpose(x_t)) + np.matmul(self.B.transpose(), [u_t])
        return x_t_1

    def reset_env(self, x0, epi_time):
        ## initialises environment
        assert np.shape(x0) == (2,)
        self.t = 0
        self.sim_data = np.zeros((4, int(epi_time/self.dt)+1))
        assert (np.linalg.norm([x0[0], self.h]) < self.max_leg_length)
        self.sim_data[:,0][0:2] = x0
        self.sim_data[:,0][2] = 0
        self.sim_data[:,0][3] = self.h
        
        return self.sim_data[:,0][0:2]
        
    def step_env(self, u, step_time):
        ## integrates the simulation for the duration of one step time(actual step not time step)
        ## This way we can look at things as snap shots of steps not worrying about things in between
        for i in range(int(step_time/self.dt)):
            self.sim_data[:,self.t + 1][0:2] = self.integrate_lip_dynamics(self.sim_data[:,self.t][0:2],\
                                                       self.sim_data[:,self.t][2])
            self.sim_data[:,self.t + 1][2] = self.sim_data[:,self.t][2] #u0
            self.sim_data[:,self.t + 1][3] = self.sim_data[:,self.t][3] #h
            self.t += 1
        
        self.sim_data[:,self.t][2] += self.action_space[u] # new u
                
        return self.sim_data[:,self.t][0:2], self.compute_cost(), self.isdone()
        
    def isdone(self):
        ## checks if the kinematic constraint is violated and terminates episode
        current_leg_length = np.linalg.norm([self.sim_data[:,self.t][0] - self.sim_data[:,self.t][2], self.h])
        if current_leg_length > self.max_leg_length:
            return True
        else:
            return False
    
    def compute_cost(self):
        '''
        computes cost: 
            which is square of distance between u and current com + 
            velocity of Com + square of step length 
        '''
        cost = abs(self.sim_data[:,self.t][0] - self.sim_data[:,self.t][2]) #|x0 - u0|
        cost += abs(self.sim_data[:,self.t][1]) #|xd|
        cost += abs(self.sim_data[:,self.t][2] - self.sim_data[:,self.t-1][2]) # |u|
        
        return cost
    
    def random_action(self):
        ## generates a random action to input to the simulation
        action = np.random.randint(len(self.action_space))
        return action
    
    def show_episode(self, freq, i_no):
        ## Input:
            ## Freq : frame rate (if freq = 5 one in 5 is shown)
            ## i_no : iteration number 
        sim_data = self.sim_data[:,::freq]

        fig = plt.figure()
        ax = plt.axes(xlim=(-2, 2), ylim=(0, sim_data[:,0][3] + 0.05))
        text_str = "iter - " + str(i_no)
        line, = ax.plot([], [], lw=3)
        def init():
            line.set_data([], [])
            return line,
        def animate(i):
            x = sim_data[:,i][0]
            y = sim_data[:,i][3]
            u = sim_data[:,i][2]
            line.set_data([u,x], [0,y])
            return line,
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        ax.text(0.05, 0.95, text_str, transform=ax.transAxes, fontsize=15,
        verticalalignment='top', bbox=props)
        
        anim = FuncAnimation(fig, animate, init_func=init,
                                       frames=np.shape(sim_data)[1], interval=25, blit=True)

        plt.close(fig)
        plt.close(anim._fig)
        IPython.display.display_html(IPython.core.display.HTML(anim.to_html5_video()))
    

In [64]:
## env test 
no_steps = 10 ## simulates 10 steps taken by the inverted pendulum
step_time = 0.1 ## duration after which step is taken by pendulum
env = lipm_env(0.2, 0.22)
state = env.reset_env([0.2,0], no_steps*step_time)
for t in range(no_steps):
    action = env.random_action()
    next_state, cost, done = env.step_env(action, step_time)
    print(state, action, cost, next_state, done)
    state = next_state
    
env.show_episode(5, 1)


[0.2 0. ] 3 1.4210771236987787 [0.25047467 1.06060246] True
[0.25047467 1.06060246] 0 3.88544090545559 [0.44223475 2.94820616] True
[0.44223475 2.94820616] 3 8.822748751231423 [0.94198928 7.49575947] True
[0.94198928 7.49575947] 4 18.53626922789453 [ 2.07340514 16.13286408] True
[ 2.07340514 16.13286408] 0 38.143811638122486 [ 4.42415576 32.94965587] True
[ 4.42415576 32.94965587] 1 77.7651126812842 [ 9.24183299 67.64327969] True
[ 9.24183299 67.64327969] 4 157.29873386691224 [ 19.06788611 137.51584775] True
[ 19.06788611 137.51584775] 0 317.2130748145215 [ 38.92800522 277.13006959] True
[ 38.92800522 277.13006959] 6 638.3494388085027 [ 78.95012163 558.46431718] True
[ 78.95012163 558.46431718] 1 1283.0705530461473 [ 159.46128105 1122.45427199] True


Your browser does not support the video tag.